<a href="https://colab.research.google.com/github/gokcergun/Code_examples/blob/main/Road_Sign_FasterRCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For Faster RCNN we will use MMDetection implmentation.
<a href="https://github.com/open-mmlab/mmdetection">MMDetection</a> is an open source object detection toolbox based on PyTorch. It is a part of <a href="https://openmmlab.com">OpenMMLab</a> consists of more than 100 pre-trained models to use for fine-tuning for object detection and instance segmentation tasks. 

# 1. Set-up


### Connect GoogleDrive
Data_path: "/content/drive/MyDrive/Datasets/data_sign_coco"

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


### Install MMDetection

In [2]:
# Check GPU setting of GoogleColabs
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [3]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
!pip install mmcv-full

# Install mmdetection
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection

!pip install -e .

# install Pillow 7.0.0 back in order to avoid bug in colab
!pip install Pillow==7.0.0

Looking in links: https://download.pytorch.org/whl/torch_stable.html
Cloning into 'mmdetection'...
remote: Enumerating objects: 22609, done.
remote: Total 22609 (delta 0), reused 0 (delta 0), pack-reused 22609
Receiving objects: 100% (22609/22609), 25.57 MiB | 13.99 MiB/s, done.
Resolving deltas: 100% (15830/15830), done.
/content/mmdetection
Obtaining file:///content/mmdetection
  Running setup.py develop for mmdet


In [4]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.5.1+cu101 True
2.20.0
11.1
GCC 7.5


In [5]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.5.1+cu101 True
2.20.0
11.1
GCC 7.5


## Data Format
MMDetection library requires the annotations as json files in COCO format with the following necessary keys:
```
'images': [
    {
        'file_name': 'COCO_val2014_000000001268.jpg',
        'height': 427,
        'width': 640,
        'id': 1268
    },
    ...
],

'annotations': [
    {
        'segmentation': [[192.81,
            247.09,
            ...
            219.03,
            249.06]],  # if you have mask labels
        'area': 1035.749,
        'iscrowd': 0,
        'image_id': 1268,
        'bbox': [192.81, 224.8, 74.73, 33.43],
        'category_id': 16,
        'id': 42986
    },
    ...
],

'categories': [
    {'id': 0, 'name': 'car'},
 ]
```




## RoadSignDataset  
We first converted our dataset to COCO format using <a href=https://roboflow.com/formats/coco-json>Roboflow</a>. We used the same train, test, validation split that was used in Yolov5 model. 

We have train, test, validation folders. Each involves the images in jpg format and a json file for annotations. 

In [6]:
import os 
os.listdir("/content/drive/MyDrive/Datasets/data_sign_coco")

['results_fasterrcnn',
 'results_old',
 'annotations',
 'test_data',
 'train_data',
 'val_data']

In [7]:
# read json file within train folder
import json
#read json files
open_file = open("/content/drive/MyDrive/Datasets/data_sign_coco/annotations/instances_train.json")
annotations_train = json.load(open_file)
open_file.close()

In [8]:
print("annotation keys:", annotations_train.keys())
print("images example:", annotations_train["images"][0])
print("annotations example:", annotations_train["annotations"][0])
print("categories example:", annotations_train["categories"])

annotation keys: dict_keys(['info', 'licenses', 'categories', 'images', 'annotations'])
images example: {'id': 0, 'license': 1, 'file_name': 'road376_png.rf.5a4e06e629169a0a8f300240e7a54d1d.jpg', 'height': 416, 'width': 416, 'date_captured': '2022-01-08T00:16:02+00:00'}
annotations example: {'id': 0, 'image_id': 0, 'category_id': 2, 'bbox': [54, 201, 134.5, 100], 'area': 13450, 'segmentation': [], 'iscrowd': 0}
categories example: [{'id': 0, 'name': 'road-signs', 'supercategory': 'none'}, {'id': 1, 'name': 'crosswalk', 'supercategory': 'road-signs'}, {'id': 2, 'name': 'speedlimit', 'supercategory': 'road-signs'}, {'id': 3, 'name': 'stop', 'supercategory': 'road-signs'}, {'id': 4, 'name': 'trafficlight', 'supercategory': 'road-signs'}]


## Modify Config
In the next step, we need to modify the config for the training. To accelerate the process, we finetune a detector using a pre-trained detector.

In [10]:
# get current working directory 
os.getcwd()

'/content/mmdetection'

We will use faster_rcnn_r50_fpn_1x_coco implementation of Faster RCNN in the implementation library of mmdetection. We will load its config file and make the required changes to use it with a custom dataset. 

In [11]:

from mmcv import Config
cfg= Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')

In [11]:
# Some default values that we will adjust to our custom dataset. 
print(cfg.keys())
print(cfg.data_root)
print(cfg.data.train.ann_file)
print(cfg.data.train.img_prefix)
print(cfg.data.val.ann_file)
print(cfg.data.val.img_prefix) 
print(cfg.data.test.ann_file) 
print(cfg.data.test.img_prefix)
print(cfg.log_config.hooks[0].type)

dict_keys(['model', 'dataset_type', 'data_root', 'img_norm_cfg', 'train_pipeline', 'test_pipeline', 'data', 'evaluation', 'optimizer', 'optimizer_config', 'lr_config', 'runner', 'checkpoint_config', 'log_config', 'custom_hooks', 'dist_params', 'log_level', 'load_from', 'resume_from', 'workflow'])
data/coco/
data/coco/annotations/instances_train2017.json
data/coco/train2017/
data/coco/annotations/instances_val2017.json
data/coco/val2017/
data/coco/annotations/instances_val2017.json
data/coco/val2017/
TextLoggerHook


In [12]:
# change config parameters
from mmdet.apis import set_random_seed

cfg["classes"] = ("road-signs", "crosswalk", "speedlimit", "stop", "trafficlight")
# define the data root
cfg.data_root = "/content/drive/MyDrive/Datasets/data_sign_coco/"
# define paths for train images and annotations
cfg.data.train.ann_file = '/content/drive/MyDrive/Datasets/data_sign_coco/annotations/instances_train.json'
cfg.data.train.img_prefix = '/content/drive/MyDrive/Datasets/data_sign_coco/train_data/'
cfg.data.train.classes = cfg.classes
# define paths for validation images and annotations
cfg.data.val.ann_file = '/content/drive/MyDrive/Datasets/data_sign_coco/annotations/instances_val.json'
cfg.data.val.img_prefix = '/content/drive/MyDrive/Datasets/data_sign_coco/val_data/'
cfg.data.val.classes = cfg.classes
# define paths for test images and annotations
cfg.data.test.ann_file = '/content/drive/MyDrive/Datasets/data_sign_coco/annotations/instances_test.json'
cfg.data.test.img_prefix = '/content/drive/MyDrive/Datasets/data_sign_coco/test_data/'
cfg.data.test.classes = cfg.classes
# change number of classes to 5 (default is 80)
cfg.model.roi_head.bbox_head.num_classes = 5
#cfg.model.roi_head.mask_head.num_classes = 5
#cfg.model.bbox_head.num_classes = 5
# load pre-trained model
cfg.load_from = "https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth"

# Set up working dir to save files and logs.
cfg.work_dir = '/content/drive/MyDrive/Datasets/data_sign_coco/results_fasterrcnn'

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.optimizer.lr = 0.02 / 8
cfg.lr_config.warmup = None
cfg.log_config.interval = 10


# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = 'bbox'
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 3
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 3

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

# tensorboard login
#cfg.log_config.hooks[0].type='TensorboardLoggerHook'

#batch size
cfg.data.samples_per_gpu = 3
cfg.lr_config.policy = "step"

#number of epochs
cfg.runner.max_epochs = 18


# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')


Config:
model = dict(
    type='FasterRCNN',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet50')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_w

In [13]:
print(cfg.keys())
print(cfg.data_root)
print(cfg.data.train.ann_file)
print(cfg.data.train.img_prefix)
print(cfg.data.val.ann_file)
print(cfg.data.val.img_prefix) 
print(cfg.data.test.ann_file)
print(cfg.data.test.img_prefix)
print(cfg.model.roi_head.bbox_head.num_classes)
print(cfg.workflow)

dict_keys(['model', 'dataset_type', 'data_root', 'img_norm_cfg', 'train_pipeline', 'test_pipeline', 'data', 'evaluation', 'optimizer', 'optimizer_config', 'lr_config', 'runner', 'checkpoint_config', 'log_config', 'custom_hooks', 'dist_params', 'log_level', 'load_from', 'resume_from', 'workflow', 'classes', 'work_dir', 'seed', 'gpu_ids'])
/content/drive/MyDrive/Datasets/data_sign_coco/
/content/drive/MyDrive/Datasets/data_sign_coco/annotations/instances_train.json
/content/drive/MyDrive/Datasets/data_sign_coco/train_data/
/content/drive/MyDrive/Datasets/data_sign_coco/annotations/instances_val.json
/content/drive/MyDrive/Datasets/data_sign_coco/val_data/
/content/drive/MyDrive/Datasets/data_sign_coco/annotations/instances_test.json
/content/drive/MyDrive/Datasets/data_sign_coco/test_data/
5
[('train', 1)]


In [14]:
# Tensorboard  (optional)
%load_ext tensorboard

In [15]:
import datetime
log_dir = "/content/drive/MyDrive/Datasets/data_sign_coco/results_fasterrcnn" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# Clear out any prior log data.
!rm -rf log_dir/tf_logs

# Train new detector - with pre-trained model

In [16]:
import datetime
starttime = str(datetime.datetime.now())
print("starting time: ", starttime)

starting time:  2022-01-16 14:57:03.802903


In [17]:
import tensorflow as tf
import datetime
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
import mmcv
import os.path as osp

# Build dataset
datasets = [build_dataset(cfg.data.train)]

log_dir = "/content/drive/MyDrive/Datasets/data_sign_coco/results_fasterrcnn" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


# Build the detector
model = build_detector(
    cfg.model)

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


2022-01-16 14:57:10,467 - mmdet - INFO - load checkpoint from http path: https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
2022-01-16 14:57:10,589 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([6, 1024]).
size mismatch for roi_head.bbox_head.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([6]).
size mismatch for roi_head.bbox_head.fc_reg.weight: copying a param with shape torch.Size([320, 1024]) from checkpoint, the shape in current model is torch.Size([20, 1024]).
size mismatch for roi_head.bbox_head.fc_reg.bias: copying a param with shape torch.Size([320]) from checkpoint, the shape in current model is torch.Size([20]).
2022-01-16 14:57:10,593 - m

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


2022-01-16 14:57:17,145 - mmdet - INFO - Epoch [1][10/233]	lr: 2.500e-03, eta: 0:45:04, time: 0.646, data_time: 0.223, memory: 3445, loss_rpn_cls: 0.0174, loss_rpn_bbox: 0.0047, loss_cls: 0.5231, acc: 88.3464, loss_bbox: 0.1128, loss: 0.6580
2022-01-16 14:57:20,953 - mmdet - INFO - Epoch [1][20/233]	lr: 2.500e-03, eta: 0:35:43, time: 0.381, data_time: 0.015, memory: 3445, loss_rpn_cls: 0.0085, loss_rpn_bbox: 0.0041, loss_cls: 0.1742, acc: 95.2604, loss_bbox: 0.1844, loss: 0.3713
2022-01-16 14:57:24,780 - mmdet - INFO - Epoch [1][30/233]	lr: 2.500e-03, eta: 0:32:36, time: 0.383, data_time: 0.015, memory: 3445, loss_rpn_cls: 0.0090, loss_rpn_bbox: 0.0032, loss_cls: 0.1646, acc: 94.5378, loss_bbox: 0.2189, loss: 0.3957
2022-01-16 14:57:28,621 - mmdet - INFO - Epoch [1][40/233]	lr: 2.500e-03, eta: 0:31:02, time: 0.384, data_time: 0.014, memory: 3446, loss_rpn_cls: 0.0026, loss_rpn_bbox: 0.0038, loss_cls: 0.0878, acc: 96.9857, loss_bbox: 0.1400, loss: 0.2342
2022-01-16 14:57:32,482 - mmdet 

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 88/88, 2.5 task/s, elapsed: 35s, ETA:     0s

...

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 88/88, 15.7 task/s, elapsed: 6s, ETA:     0s

2022-01-16 15:25:50,060 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.05s).
Accumulating evaluation results...


2022-01-16 15:25:50,157 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.725
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.915
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.827
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.398
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.829
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.877
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.786
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.786
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.786
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.446
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.881
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000

DONE (t=0.04s).


In [18]:
endtime = str(datetime.datetime.now())
print("ending time: ", endtime)

ending time:  2022-01-16 15:25:51.176765


In [14]:
!python tools/test.py configs/myconfig_fastercnn.py /content/drive/MyDrive/Datasets/data_sign_coco/results_fasterrcnn/latest.pth --eval bbox


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
load checkpoint from local path: /content/drive/MyDrive/Datasets/data_sign_coco/results_fasterrcnn/latest.pth
[>>] 88/88, 16.5 task/s, elapsed: 5s, ETA:     0s
Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.04s).
Accumulating evaluation results...
DONE (t=0.03s).

 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.739
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.911
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.837
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.570
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.828
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.814
 Average Recall     (AR) @[ IoU=0.50:0.9

In [24]:
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
#img = "/content/drive/MyDrive/Datasets/data_sign_coco/test_data/road261_png.rf.770512ea3f01dcaa420cf62c24cf3347.jpg"
model.cfg = cfg
images = os.listdir('/content/drive/MyDrive/Datasets/data_sign_coco/test_data')
test_images = []
for img in images:
  img_path = os.path.join("/content/drive/MyDrive/Datasets/data_sign_coco/test_data", img)
  test_images.append(img_path)

RuntimeError: ignored

In [26]:
test_images

['/content/drive/MyDrive/Datasets/data_sign_coco/test_data/road261_png.rf.770512ea3f01dcaa420cf62c24cf3347.jpg',
 '/content/drive/MyDrive/Datasets/data_sign_coco/test_data/road498_png.rf.0e7020b45f01a14cd1cb9d1bbdc99766.jpg',
 '/content/drive/MyDrive/Datasets/data_sign_coco/test_data/road100_png.rf.00cc961633005fae475e1e9633c0446f.jpg',
 '/content/drive/MyDrive/Datasets/data_sign_coco/test_data/road50_png.rf.cb0c3b23a6b712005e31176df4892db8.jpg',
 '/content/drive/MyDrive/Datasets/data_sign_coco/test_data/road709_png.rf.6c7c19a7d30b0669ebf0b8456b2775b0.jpg',
 '/content/drive/MyDrive/Datasets/data_sign_coco/test_data/road558_png.rf.634589445c11561d7f985158dc6a0b2b.jpg',
 '/content/drive/MyDrive/Datasets/data_sign_coco/test_data/road445_png.rf.188a41709a254d6fa8aee63331184d57.jpg',
 '/content/drive/MyDrive/Datasets/data_sign_coco/test_data/road617_png.rf.6de13550e40770e1cbdba3702080110c.jpg',
 '/content/drive/MyDrive/Datasets/data_sign_coco/test_data/road367_png.rf.8aa56831b4bed9e00482732

In [ ]:
# show the results
show_result_pyplot(model, img, result)

AttributeError: ignored

In [ ]:
# test a single image
img = 'demo.jpg'
result = inference_detector(model, img)